<a href="https://colab.research.google.com/github/divassya/BigDataAnalysis/blob/main/AssiyaKaratay_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Info 
Assignment 1
MET CS777 Big Data Analytics

Faculty - Farshid Alizadeh-Shabdiz, PhD, MBA

Student - Assiya Karatay U95161396 karatay@bu.edu 857-294-7028

#### import libraries

In [1]:
# Spark installation on Colab
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# !tar xf spark-3.0.1-bin-hadoop3.2.tgz
# !pip install -q findspark
# !rm -rf spark-3.0.1-bin-hadoop3.2.tgz

!pip install --ignore-installed -q pyspark==3.1.2 

     |████████████████████████████████| 212.4 MB 94 kB/s 
     |████████████████████████████████| 198 kB 74.9 MB/s 


In [2]:
# Set JAVA_HOME and SPARK_HOME
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "spark-3.0.1-bin-hadoop3.2"

# import findspark
# findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME


import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

#### get the data

In [6]:
# get the data
path = "./"
filename = "taxi-data-sorted-small.csv.bz2"
# filename = sys.argv[1]
lines = sc.textFile(filename)
taxilines = lines.map(lambda x: x.split(','))

#### data processing

In [7]:
# Exception handling and removing wrong data lines
def isfloat(value):
  try:
    float(value)
    return True
  except:
    return False

# remove lines if they do not have 17 values
def correctRows(p):
  if (len(p) == 17):
    if (isfloat(p[5]) and isfloat(p[11])):
      if (float(p[5])!=0 and float(p[11])!=0):
        return p

#cleaning up data
taxilinesCorrected = taxilines.filter(correctRows)

#### Task 1 - Top-10 Active Taxis (25 points) 

In [8]:
# the top ten taxis that have had the largest number of drivers. 
# p[0] is the medallion of the taxi and p[1] is the hack license of the driver

# 1. get a pair of distinct medallions and drivers
TaxiDriver = taxilinesCorrected.map(lambda p: (p[0], p[1]))
distinctTaxiDriver = TaxiDriver.distinct()
# 2. count number of drivers for each taxi 
countDriversPerTaxi = distinctTaxiDriver.map(lambda p: (p[0], 1)).reduceByKey(lambda a,b: a+b)
# 3. Get the answer in a pair of the medallion and the number of drivers) 
top10Taxis = sc.parallelize(countDriversPerTaxi.top(10, lambda x:x[1]))
# 4. Save it to text file
output_file = 'ActiveTaxis'
# output_file = sys.argv[2]
top10Taxis.saveAsTextFile(output_file)

# sc.stop()  

#### Task 2 - Top-10 Best Drivers (35 Points) 

In [9]:
# 1. Map hack licenses,trip time in minutes and total amount of revenue  
minsAndRevenue = taxilinesCorrected.map(lambda p: (p[1], (float(p[4])/60, float(p[16]))))
# 2. Sum minutes and revenues per hack license
sumOfMinsAndRevenue = minsAndRevenue.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
# 3. Filter out trips that have less than 1 minute and 0 revenue
zerosRemoved = sumOfMinsAndRevenue.filter(lambda x:x[1][0] >= 1 and x[1][1] != 0)
# 4. Divide the total revenue by total minutes for each hack license
moneyPerMinute = zerosRemoved.map(lambda a: (a[0], a[1][1]/a[1][0]))
# 5. Get top 10 drivers with their revenue per minute) 
top10Drivers = sc.parallelize(moneyPerMinute.top(10, lambda x:x[1]))
# 6. Save it to text file
# top10drivers.saveAsTextFile(sys.argv[2])
top10Drivers.saveAsTextFile("bestDrivers1")

# sc.stop() 

#### Task 3 - Best time of the day to Work on Taxi (40 Points) 

In [10]:
# 1. Map pickup time, miles and surcharge amount 
hourSurchargeMiles = taxilinesCorrected.map(lambda p: (p[2].split(' ')[1].split(':')[0], (float(p[5]), float(p[12]))))
# 2. Remove the trips with 0 miles and surcharge
zerosRemoved = hourSurchargeMiles.filter(lambda x: x[1][0] != 0 and x[1][1] != 0)
# 3. Sum surcharge amount and miles per hour
totalSurchargeAndMiles = zerosRemoved.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
# 4. Divide total surcharge amount by total miles for each hour
profitRatio = totalSurchargeAndMiles.map(lambda a: (a[0], a[1][1]/a[1][0]))
# 5. Get the highest ratio as (hour, profitRatio) 
highestRate = sc.parallelize(profitRatio.top(1, lambda x:x[1]))
# 6. Save the answer to text file
# highestRate.saveAsTextFile(sys.argv[2])
highestRate.saveAsTextFile("bestTaxiTime1")


sc.stop() 

#### Task 4 – Advanced question (10 points)
Here are two further tasks for advanced groups. 
•	What percentage of taxi customers pay with cash and what percentage use electronic cards? Analyze these payment methods for different time of the day and provide a list of percentages for each hour of the day? As a result provide two numbers for total percentages and a list like (hour of the day, percent paid card) 


•	We would like to measure the efficiency of taxis drivers by finding out their average earned money per mile. (Consider the total amount which includes tips, as their earned money) Implement a Spark job that can find out the top-10 efficient taxi divers. 
#### Task 4 – More Advanced questions for interested students – needs research (No Grade)
•	What are mean, median, first and third quantiles of tip amount? How do find the median? 
•	Using the IQR outlier detection method find out the top-10 outliers. 
